In [1]:
import torch

In [18]:
import os
import torch

# Пути к папкам
embeds_root = "/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid_fixed_embeds"
orig_root = "/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/cached_latents/vit_0_1"

# Собираем все пути к .pt файлам в embeds (с учетом иерархии)
embeds_pt_files = []
for root, dirs, files in os.walk(embeds_root):
    for file in files:
        if file.endswith(".pt"):
            embeds_pt_files.append(os.path.join(root, file))

print(f"Найдено файлов в embeds: {len(embeds_pt_files)}")

not_allclose = []
allclose = []
not_found_in_orig = []

for embed_path in embeds_pt_files:
    # Имя файла без пути
    fname = os.path.basename(embed_path)
    # В orig названия файлов без расширения .nii
    # Например: если embed_path = .../xxx.nii.pt, то в orig это xxx.pt
    if fname.endswith(".nii.pt"):
        orig_fname = fname.replace(".nii.pt", ".pt")
    else:
        orig_fname = fname
    orig_path = os.path.join(orig_root, orig_fname)
    if not os.path.exists(orig_path):
        not_found_in_orig.append(fname)
        continue
    try:
        t1 = torch.load(embed_path, map_location="cpu")
        import torch.nn.functional as F
        t1 = F.normalize(t1, dim=-1)
        t2 = torch.load(orig_path, map_location="cpu")
        if torch.allclose(t1.cpu(), t2.cpu(), atol=1e-5, rtol=1e-3):
            allclose.append(fname)
        else:
            not_allclose.append(fname)
    except Exception as e:
        print(f"Ошибка при сравнении {fname}: {e}")
        not_allclose.append(fname)

print(f"Совпадают (allclose): {len(allclose)}")
print(f"Не совпадают: {len(not_allclose)}")
print(f"Нет в orig: {len(not_found_in_orig)}")

if not_allclose:
    print("Не совпадающие файлы:")
    for f in not_allclose:
        print(f)
if not_found_in_orig:
    print("Файлы, которых нет в orig:")
    for f in not_found_in_orig:
        print(f)


Найдено файлов в embeds: 319


Совпадают (allclose): 319
Не совпадают: 0
Нет в orig: 0


In [5]:
a = torch.load('/home/borntowarn/projects/chest-diseases/training/weights/CT-CLIP/CT_LiPro_v2.pt')

In [7]:
a.keys()

odict_keys(['trained_model.temperature', 'trained_model.text_transformer.embeddings.position_ids', 'trained_model.text_transformer.embeddings.word_embeddings.weight', 'trained_model.text_transformer.embeddings.position_embeddings.weight', 'trained_model.text_transformer.embeddings.token_type_embeddings.weight', 'trained_model.text_transformer.embeddings.LayerNorm.weight', 'trained_model.text_transformer.embeddings.LayerNorm.bias', 'trained_model.text_transformer.encoder.layer.0.attention.self.query.weight', 'trained_model.text_transformer.encoder.layer.0.attention.self.query.bias', 'trained_model.text_transformer.encoder.layer.0.attention.self.key.weight', 'trained_model.text_transformer.encoder.layer.0.attention.self.key.bias', 'trained_model.text_transformer.encoder.layer.0.attention.self.value.weight', 'trained_model.text_transformer.encoder.layer.0.attention.self.value.bias', 'trained_model.text_transformer.encoder.layer.0.attention.output.dense.weight', 'trained_model.text_transfo

In [9]:
[i for i in a.keys() if "to_visual_latent" in i or "visual_transformer" in i]

['trained_model.visual_transformer.spatial_rel_pos_bias.net.0.0.weight',
 'trained_model.visual_transformer.spatial_rel_pos_bias.net.0.0.bias',
 'trained_model.visual_transformer.spatial_rel_pos_bias.net.1.0.weight',
 'trained_model.visual_transformer.spatial_rel_pos_bias.net.1.0.bias',
 'trained_model.visual_transformer.spatial_rel_pos_bias.net.2.weight',
 'trained_model.visual_transformer.spatial_rel_pos_bias.net.2.bias',
 'trained_model.visual_transformer.to_patch_emb_first_frame.1.weight',
 'trained_model.visual_transformer.to_patch_emb_first_frame.1.bias',
 'trained_model.visual_transformer.to_patch_emb_first_frame.2.weight',
 'trained_model.visual_transformer.to_patch_emb_first_frame.2.bias',
 'trained_model.visual_transformer.to_patch_emb_first_frame.3.weight',
 'trained_model.visual_transformer.to_patch_emb_first_frame.3.bias',
 'trained_model.visual_transformer.to_patch_emb.1.weight',
 'trained_model.visual_transformer.to_patch_emb.1.bias',
 'trained_model.visual_transformer.t

In [2]:
import torch

a1 = torch.load('/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/train_fixed_embeds_not_normalized_base/train_1/train_1_a/train_1_a_2.nii.pt')
a2 = torch.load('/home/borntowarn/projects/chest-diseases/backup/dataset/train_fixed_embeds_not_normalized_base/train_1/train_1_a/train_1_a_2.nii.pt')

torch.allclose(a1, a2, atol=1e-6, rtol=1e-6)


True

In [31]:
import os
import torch

# Пути к папкам с тензорами
base_dir = "/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/cached_latents"
vit_dir = os.path.join(base_dir, "vit_0_1")
orig_dir = os.path.join(base_dir, "vit_final1")

# Получаем список файлов в обеих папках
vit_files = set(os.listdir(vit_dir))
orig_files = set(os.listdir(orig_dir))

# Находим общие файлы
common_files = vit_files & orig_files

print(f"Общее количество файлов для сравнения: {len(common_files)}")

not_allclose = []
allclose = []
missing_in_vit = orig_files - vit_files
missing_in_orig = vit_files - orig_files

for fname in sorted(common_files):
    vit_path = os.path.join(vit_dir, fname)
    orig_path = os.path.join(orig_dir, fname)
    try:
        vit_tensor = torch.load(vit_path, map_location="cpu")
        orig_tensor = torch.load(orig_path, map_location="cpu")
        # Приводим к одному типу, если нужно
        if vit_tensor.dtype != orig_tensor.dtype:
            orig_tensor = orig_tensor.to(vit_tensor.dtype)
        # Проверяем форму
        if vit_tensor.shape != orig_tensor.shape:
            print(f"Файл {fname}: разные формы тензоров: {vit_tensor.shape} vs {orig_tensor.shape}")
            not_allclose.append(fname)
            continue
        # Проверяем на allclose
        if torch.allclose(vit_tensor, orig_tensor, atol=1e-6, rtol=1e-6):
            allclose.append(fname)
        else:
            print(f"Файл {fname}: тензоры НЕ совпадают (allclose=False)")
            not_allclose.append(fname)
    except Exception as e:
        print(f"Ошибка при обработке файла {fname}: {e}")
        not_allclose.append(fname)

print(f"\nСовпадающих тензоров (allclose=True): {len(allclose)}")
print(f"Не совпадающих тензоров (allclose=False): {len(not_allclose)}")
if missing_in_vit:
    print(f"Файлы, которых нет в vit: {missing_in_vit}")
if missing_in_orig:
    print(f"Файлы, которых нет в orig: {missing_in_orig}")

if not_allclose:
    print("\nСписок файлов с несовпадающими тензорами:")
    for fname in not_allclose:
        print(fname)
else:
    print("\nВсе тензоры совпадают (allclose=True)!")


Общее количество файлов для сравнения: 80
Файл valid_147_a_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_147_a_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_153_a_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_153_a_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_a_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_a_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_b_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_b_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_c_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_c_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_d_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_163_d_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_166_a_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_166_a_2.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_16_a_1.pt: тензоры НЕ совпадают (allclose=False)
Файл valid_16_a_2.pt: тензоры НЕ совпадают (

In [8]:
import pandas as pd

df = pd.read_csv("/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/metadata/validation_metadata.csv")

In [19]:
import numpy as np
from pathlib import Path

all_data = []
for path in Path('/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/minmax').glob('*.npz'):
    data = np.load(path)
    all_data.append(data['img_data'])

all_data = np.array(all_data)
all_data.min(), all_data.max()


(np.float64(-32767.0), np.float64(32767.0))

In [9]:
df['RescaleIntercept'].value_counts()

RescaleIntercept
-1024    2189
-8192     850
Name: count, dtype: int64

In [10]:
df['RescaleSlope'].value_counts()


RescaleSlope
1    3039
Name: count, dtype: int64